<a href="https://colab.research.google.com/github/wilberj88/Price-Models/blob/main/2_agrupacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/paular143/images_notebooks/master/logo-sin-nombre.png" alt="Universidad de Los Andes" style="width: 90px;" align="left" frameborder="300"/>

**UNIVERSIDAD DE LOS ANDES** <br>
**FACULTAD DE ECONOMÍA - EDUCACIÓN CONTINUADA** <br>
**Análisis espacial de datos y sus aplicaciones en Python 2020** <br>
**PROFESORES:** Germán González y Mateo Dulce <br><br>


# Clase 8: Dbscan


In [ ]:
# !pip3 install shapely
# !pip3 install pyproj
# !pip3 install folium
# !pip3 install pandas
# !pip3 install requests
# !pip3 install geopy
# !pip3 install geojson
# !pip3 install xlrd
# !pip3 install sklearn

import pyproj
from functools import partial
import shapely.geometry
from shapely.ops import transform
from shapely.geometry.polygon import Polygon 
from shapely.geometry import Point, MultiPoint 
from geopy.distance import great_circle
import folium
import geojson
import json
import pandas as pd
import re
import geopy
import requests
import time
import numpy as np
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from sklearn.cluster import DBSCAN



## Comercios de bogotá: Camara de Comercio 2016 - Submuestra 250 comercios - Usaquen

In [ ]:
Lugares_usaquen_sample = pd.read_pickle('/Users/germangonzalez/Dropbox (Quantil)/Uniandes Analisis espacial/Clases/Clase 8 - Dash/Lugares_sample.gzip',compression='gzip')

In [ ]:
Lugares_usaquen_sample

,ID,Nombre,Actividad,Direccion,Latitude,Longitude,Point,Grupo
0,0,TU TIENDA . U,Alojamiento y servicios de comida,CL 131 No 58 35 IN 3 AP 310,4.777510,-74.041255,POINT (-74.04125490482421 4.777510416416243),0
1,1,SERVICIOS INTEGRALES DE LOS ANDES,Actividades de atencionde la salud humana y de...,CLL 184 NO. 41-61 INT 1 APTO 402,4.760924,-74.043020,POINT (-74.04302011142549 4.760923651396257),1
2,2,VELO'S BICYCLE,Construccion,CL 164 A . 8 C - 47,4.740671,-74.027962,POINT (-74.02796248415774 4.740670707074391),2
3,3,AGROPECUARIA PIGS CATTLE TROPICAL,"Agricultura, ganader¡a, caza, silvicultura y p...",CR 55 No 151 69,4.720658,-74.039057,POINT (-74.03905669588293 4.720657536006083),3
4,4,PUNTO DE VENTA CALLE 150,Industrias manufactureras,AV 19 N.150 - 04,4.731748,-74.044676,POINT (-74.04467582597005 4.731747614581696),3
...,...,...,...,...,...,...,...,...
245,245,CORPO BELLO,Actividades de atencionde la salud humana y de...,CL 127 NO. 19 A 44,4.706272,-74.050034,POINT (-74.05003421824934 4.706272005032837),4
246,246,DENTAL CLINIC COLOMBIA,Actividades de atencionde la salud humana y de...,CR 14 A . 112 96 CONS 402,4.694596,-74.039045,POINT (-74.03904480218857 4.694596429824847),4
247,247,RISTORANTE LA BRUSCHETA,Alojamiento y servicios de comida,DG 109 NO. 14-61,4.691747,-74.043599,POINT (-74.04359933434091 4.691746978251736),4
248,248,TO BLOSSOM,Otras actividades de servicios,CR 19 . 122 49 LC 6,4.702478,-74.049456,POINT (-74.0494559466498 4.702477665611049),4


## Visualización Mapa

In [ ]:
mapa = folium.Map(location=[4.729494,-74.049345],zoom_start=15,tiles='cartodbpositron')

for j in range(0,len(Lugares_usaquen_sample)):
    punto = Lugares_usaquen_sample.iloc[j] 
    folium.Marker(
        [punto['Latitude'],punto['Longitude']]
    ).add_to(mapa)


In [ ]:
mapa

## Se remplazan el número de lugares dentro 

In [ ]:
####### ------ Transform coords ------- #########
coords_to_clusterize = np.array(list(pd.DataFrame(Lugares_usaquen_sample.apply(lambda x: [float(x['Latitude']),float(x['Longitude'])],axis=1))[0]))
coords_radians = np.radians(coords_to_clusterize)

# Distancia máxima (1.5 kilómetros en este ejemplo) que los 
# puntos pueden estar el uno del otro para ser considerados un grupo
Metros = 500
Kilometros_cluster = Metros/1000

#### ----- Radio del mundo ------ ######
Kilometros_radio_mundo = 6371.0088
####### ------- Se calcula el epsilon en radiales ------- ########
epsilon = Kilometros_cluster/Kilometros_radio_mundo

########## ---------- Calcular clusters  --------- ##########
Clusters =DBSCAN(eps=epsilon,algorithm='ball_tree',metric='haversine',n_jobs=-1).fit(coords_radians)
#### ------ Se extraen las etiquetas de cada lot, lat ----- ####
Etiquetas_clusters = Clusters.labels_
#### ------ Asignar la etiqueta a cada observación ----- ###### 
Lugares_usaquen_sample['Grupo'] = Etiquetas_clusters

##### -------- Contar el número de observaciones por cluster ------ ##### 
Indice_clusters = pd.DataFrame(pd.value_counts(pd.DataFrame(Etiquetas_clusters)[0]).sort_index())[0]


### ------- Numero de grupos creados ---- ##### 
num_clusters = len(set(Etiquetas_clusters))




## Centroides

In [ ]:
##### -------- Crear lista con las coordenadas ----- ###### 
clusters = pd.Series([coords_to_clusterize[Etiquetas_clusters == n] for n in list(set(Etiquetas_clusters))])

####### ------ función para encontrar las coordenadas de los centroides ------- ###### 
def Obtener_centro_clusters(cluster):
    ### ------- Encontrar el centroide de la figura ------ ####### 
    Centroide = MultiPoint(cluster).centroid.coords[0]
    ###### -------- Calcular distancia a un punto real -------- ######
    ##### ----------- Encontrar distancia minima ----- ####### 
    centermost_point = min(cluster , key=lambda x: great_circle(x , Centroide).m)
    return tuple(centermost_point)



###### ------- Calculate centroid ------- #######
Coordenadas_centroides = list( map(lambda x: Obtener_centro_clusters(x),clusters))
### ----- Extraer las latitude y longitudes 
Latitudes = list(map(lambda x: x[0],Coordenadas_centroides))
Longitudes = list(map(lambda x: x[1],Coordenadas_centroides)) 
### --------- Consolidar dataframe ---- ### 
Centroides_info = pd.DataFrame({'Longitude':Longitudes, 'Latitude':Latitudes})
### ------- Buscar información del centroide ------- #### 
Centroides = Centroides_info.apply(lambda row: Lugares_usaquen_sample[(Lugares_usaquen_sample['Latitude']==row['Latitude']) & (Lugares_usaquen_sample['Longitude']==row['Longitude'])].iloc[0], axis=1)
Centroides = pd.DataFrame(Centroides)



### Crear Mapa

In [ ]:

mapa = folium.Map(location=[Centroides.head(1)['Latitude'].iloc[0],Centroides.head(1)['Longitude'].iloc[0]],zoom_start=13,tiles='cartodbpositron')

Vector_colores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred','lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

##### ------- Graficar Centroides ------ ###### 
for j in range(0,len(Centroides)):
    punto = Centroides.iloc[j] 
    folium.Marker(
        [punto['Latitude'],punto['Longitude']],icon=folium.Icon(color=Vector_colores[j], icon='cloud'),tooltip=punto['Nombre']
    ).add_to(mapa)
    
    
    ##### ------- Graficar vecinos de los centroides ------ ###### 
    Vecinos_cluster = Lugares_usaquen_sample[Lugares_usaquen_sample['Grupo'] == punto['Grupo']]
    
    ##### ------- Excluir al centro ------ ####
    Vecinos_cluster = Vecinos_cluster[Vecinos_cluster['ID'] != punto['ID']] 
    for v in range(0,len(Vecinos_cluster)):
        punto_vecino = Vecinos_cluster.iloc[v] 
        folium.Marker(
            [punto_vecino['Latitude'],punto_vecino['Longitude']],icon=folium.Icon(color=Vector_colores[j], icon='book'),tooltip=punto_vecino['Nombre']
        ).add_to(mapa)



